In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../../src/')

import os

from aind_vr_foraging_analysis.utils.plotting import general_plotting_utils as plotting, plotting_friction_experiment as f
from aind_vr_foraging_analysis.utils.parsing import parse, AddExtraColumns, data_access

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
from pathlib import Path

sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = r'Z:\scratch\vr-foraging\data'
data_path = r'../../../data/'

color1='#d95f02'
color2='#1b9e77'
color3='#7570b3'
color4='#e7298a'
odor_list_color = [color1, color2, color3]
color_dict = {0: color1, 1: color2, 2: color3}
color_dict_label = {'Ethyl Butyrate': color1, 'Alpha-pinene': color2, 'Amyl Acetate': color3, 
                    '2-Heptanone' : color2, 'Methyl Acetate': color1, 'Fenchone': color3, '2,3-Butanedione': color4,
                    'Methyl Butyrate': color1}
# color_dict_label = {'Ethyl Butyrate': '#d95f02', 'Alpha-pinene': '#1b9e77', 'Amyl Acetate': '#7570b3', 
#                     '2-Heptanone' : '#1b9e77', 'Methyl Acetate': '#d95f02', 'Fenchone': '#7570b3', '2,3-Butanedione': '#e7298a'}
dict_odor = {}
rate = -0.12
offset = 0.6
dict_odor['Ethyl Butyrate'] = {'rate':rate, 'offset':offset, 'color': '#d95f02'}
dict_odor['Methyl Butyrate'] = {'rate':rate, 'offset':0.9, 'color': '#d95f02'}
dict_odor['Alpha-pinene'] = {'rate':rate, 'offset':offset, 'color': '#1b9e77'}
dict_odor['Amyl Acetate'] = {'rate':rate, 'offset':offset, 'color': '#7570b3'}
dict_odor['Methyl Acetate'] = {'rate':rate, 'offset':offset, 'color': color1}
dict_odor['2,3-Butanedione'] = {'rate':rate, 'offset':offset, 'color': color4}
dict_odor['Fenchone'] = {'rate':rate, 'offset':offset, 'color': '#7570b3'}
dict_odor['2-Heptanone'] = {'rate':rate, 'offset':offset, 'color': '#7570b3'}

# Define exponential function
def exponential_func(x, a, b):
    return a * np.exp(b * x)

def format_func(value, tick_number):
    return f"{value:.0f}"

results_path = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\experiments\batch 4 - manipulating cost of travelling and global statistics\results'



### Generating the dataset

In [ ]:
#Don't change this date
date = datetime.datetime.strptime("4/25/2024", "%m/%d/%Y").date()
date_final = datetime.datetime.strptime("7/01/2024", "%m/%d/%Y").date()

In [ ]:
summary_df = pd.DataFrame()
all_epochs = pd.DataFrame()
within_session_n = 0
experiment = 0
for mouse in ["713545","713578", "707349", "716455", "716458","715865","715866","715869","715867","715870","694569","716456","716457"]:
    print(mouse)
    session_n = 0
    directory = os.path.join(base_path, mouse)
    files = os.listdir(os.path.join(base_path, mouse))
    sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=False)
    for file_name in sorted_files:
        print(file_name)
        session_path = os.path.join(base_path, mouse, file_name)
        session = str(session_path)[-22:]   
        session_path = Path(session_path)
        
        if datetime.date.fromtimestamp(os.path.getctime(session_path)) < date or datetime.date.fromtimestamp(os.path.getctime(session_path)) > date_final:
            continue
        
        try:
            data = parse.load_session_data(session_path)
        except:
            print('Error loading session data')
            continue
        
        if 'TaskLogic' in data['config'].streams.keys():
            tasklogic = 'TaskLogic'
        else:
            tasklogic = 'tasklogic_input'
            
        data['config'].streams[tasklogic].load_from_file()
        
        if 'environment_statistics' in data['config'].streams[tasklogic].data:
            environment_statistics = 'environment_statistics'
            reward_specification = 'reward_specification'
            odor_index = 'reward_function'
        else:
            environment_settings = 'environmentStatistics'
            reward_specification = 'rewardSpecifications'
            odor_index = 'odorIndex'

        # Don't select the session if the preward is constant (that means this is not the type of experiment we want for this analysis)
        try:
            if (data['config'].streams[tasklogic].data[environment_statistics]['patches'][0][reward_specification]['reward_function']['probability']['function_type'] == 'ConstantFunction' and 
                data['config'].streams[tasklogic].data[environment_statistics]['patches'][1][reward_specification]['reward_function']['probability']['function_type'] == 'ConstantFunction'):
                print('Not good session')
                continue
            if len(data['config'].streams[tasklogic].data[environment_statistics]['patches']) != 3:
                print('Not good session')
                continue
        except:
            continue
        
        print(data['config'].streams[tasklogic].data[environment_statistics]['patches'][2]['label']
        )
        if (data['config'].streams[tasklogic].data[environment_statistics]['patches'][0][reward_specification]['reward_function']['probability']['a'] == 0.6 and 
            data['config'].streams[tasklogic].data[environment_statistics]['patches'][1][reward_specification]['reward_function']['probability']['a'] == 0.9):
            if experiment != 'base':
                within_session_n = 0
            experiment = 'base'
            within_session_n+=1

        elif data['config'].streams[tasklogic].data[environment_statistics]['patches'][2]['label'] == 'Fenchone':
            if experiment != 'experiment2':
                within_session_n = 0
            experiment = 'experiment2'
            within_session_n+=1
        else:
            if experiment != 'experiment1':
                within_session_n = 0
            experiment = 'experiment1'
            within_session_n+=1
            

        # Parse data
        try:
            all_epochs = parse.parse_dataframe(data)
        except:
            print('Error parsing data')
            continue
        
        if all_epochs.empty:
            print('Empty dataframe')
            continue
        
        
        all_epochs['session'] = session
        all_epochs['within_session_n'] = within_session_n
        all_epochs['duration_epoch'] = all_epochs.index.to_series().diff().shift(-1)
        all_epochs['mouse'] = mouse
        all_epochs['rig'] =  data['config'].streams.rig_input.data['rig_name']
        all_epochs['experiment'] = experiment
        
        session_n += 1
        all_epochs['session_n'] = session_n
        
        expanded = AddExtraColumns(all_epochs, run_on_init=True)
        all_epochs = expanded.get_all_epochs()
        reward_sites = expanded.get_odor_sites()
        
        if experiment =='experiment1' and reward_sites.reward_probability.max() == 0.6:
            environment = 'low'
        elif experiment =='experiment1' and reward_sites.reward_probability.max() == 0.9:
            environment = 'high'
        else:
            environment = 'mix'
        print(f'Good session and {experiment}', environment, session, within_session_n, session_n)
        all_epochs['environment'] = environment
        reward_sites['environment'] = environment

        reward_sites['perceived_reward_probability'] = reward_sites['after_choice_cumulative_rewards'] / (reward_sites['site_number'] +1)
        
        # Remove segments where the mouse was disengaged
        last_engaged_patch = reward_sites['patch_number'][reward_sites['skipped_count'] >= 3].min()
        if pd.isna(last_engaged_patch):
            last_engaged_patch = reward_sites['patch_number'].max()
            
        reward_sites['engaged'] = np.where((reward_sites['patch_number'] <= last_engaged_patch)|(reward_sites['patch_number'] >= 20), 1, 0)
        
        all_epochs = pd.concat([all_epochs.loc[all_epochs.label !='OdorSite'], reward_sites]).sort_index()
    
        summary_df = pd.concat([summary_df, all_epochs])


In [ ]:
summary_df.to_csv(os.path.join(data_path, 'batch_3.csv'))